In [3]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

dataset = 'Cora'
dataset = Planetoid("./data", dataset, transform=T.NormalizeFeatures())


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(dataset.num_features, 8, heads=8,
                             dropout=0.6).jittable()

        self.conv2 = GATConv(64, dataset.num_classes, heads=1, concat=True,
                             dropout=0.6).jittable()

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), dataset[0].to(device)
model = torch.jit.script(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

import sys
class Printer():
    """Print things to stdout on one line dynamically"""
    def __init__(self,num_period=10):
        self.num_period = num_period
        self.init_value = 0

    def __call__(self,data) :
        if self.init_value % self.num_period == 0 :
            print('\n'+data.__str__())
            self.init_value = 1
        else :
            sys.stdout.write("\r\x1b[K"+data.__str__())
            sys.stdout.flush()
            self.init_value += 1 
            
printf = Printer(num_period=50)

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    printf(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Epoch: 001, Loss: 1.9440, Train: 0.4786, Val: 0.2560, Test: 0.2870
Epoch: 050, Loss: 1.4688, Train: 0.9286, Val: 0.7640, Test: 0.7690
Epoch: 051, Loss: 1.4181, Train: 0.9286, Val: 0.7620, Test: 0.7650
Epoch: 100, Loss: 0.9614, Train: 0.9857, Val: 0.8180, Test: 0.8270
Epoch: 101, Loss: 0.9251, Train: 0.9857, Val: 0.8200, Test: 0.8220
Epoch: 150, Loss: 0.7430, Train: 0.9857, Val: 0.8160, Test: 0.8240
Epoch: 151, Loss: 0.7614, Train: 0.9857, Val: 0.8180, Test: 0.8250
Epoch: 200, Loss: 0.6708, Train: 0.9929, Val: 0.8020, Test: 0.8230